# Scraping for Putin's close associates
https://www.spisok-putina.org/

The website (https://www.spisok-putina.org/) **“PUTIN’S LIST”** is created by the Free Russia Forum in order **to search, consolidate and unify data** on those responsible for creating and running a system of unchecked power and suppressing civil liberties both in Russia and abroad.

The project aims to gather information related to persons of the following categories:

* The **“POWER-HOLDERS”** of Russia, who seized and unlawfully hold power in the country;
* The **“EXECUTORS”**, who, as public servants and MPs, directly implement the Kremlin’s policy of felonious retention of power in the country;
* The **“TRANSGRESSORS”**, from among the judges, prosecutors, investigators, lawyers, law enforcement officers and their agents responsible for the direct implementation of the repression and prosecution of the opposition;
* The **“AGGRESSORS”**, who are responsible for carrying out the policies of war, occupation and chaos against other countries, in violation of international law, in the political interests of the leadership of the country;
* The **“BENEFICIARIES”**, foreign partners of the Russian government who contribute to the implementation of its felonious plans to undermine international law and global peace, justice and democracy.
* The **“OLIGARCHS AND CORRUPT OFFICIALS”**, who are responsible for plundering Russia on a state-wide scale, by direct membership in the Russian government or proximity to it through relatives and other proxies;
* The **“PROPAGANDISTS”**, who are responsible for creating and disseminating state information, deliberately deceiving people in the interests of the political leadership and creating an atmosphere of widespread hatred and intolerance;
* The **“ACCOMPLICES”**, who reap personal profit from cooperation with government authorities, to the detriment of the interests of the country and its citizens;

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import re
import pandas as pd
from transliterate import translit

I started by creating a list that only keeps the first names of individuals. I used this for adding the name to the original url 
so I can access and scrape each page apart

In [28]:
base_url = "https://www.spisok-putina.org/en/personas/page/#/"

"""this list keeps only the first name"""
persons_list = []

for page_number in range(1, 141): # there are 140 webpages for each particular individual
    url = base_url.replace("#", str(page_number))
    response = requests.get(url) # send request
    soup = bs(response.content, 'html.parser')  # bs object
    person_names = soup.find_all('div', class_='preview-material__content') # extract all person names

    # append the first word of each name to the persons_list
    for person_name in person_names:
        full_name = person_name.text.strip().replace('\n', ' ').replace('  ', ' ')
        first_word = full_name.split()[0]
        persons_list.append(first_word.lower())

In [29]:
persons_list

['abakarov',
 'abdullazyanov',
 'abramov',
 'abramovich',
 'afanasieva',
 'afanasov',
 'afanasyeva',
 'afasizhev',
 'afonin',
 'agabekyan',
 'agaev',
 'agafonova',
 'agalarov',
 'agibalov',
 'aginey',
 'agramunt',
 'agranovsky',
 'ahmadov',
 'aigistov',
 'aitkulova',
 'akhmadiyeva',
 'akhmetov',
 'akimov',
 'akimov',
 'akopov',
 'aksakov',
 'aksyonenko',
 'aksyonov',
 'albakova',
 'albekov',
 'aleinik',
 'alekhin',
 'alekperov',
 'alekseenko',
 'alekseev',
 'alexandrov',
 'alexandrov',
 'alibaev',
 'alikhanov',
 'alimova',
 'almazov',
 'alshevskikh',
 'altabaeva',
 'altudov',
 'altukhov',
 'amelchakov',
 'ammosov',
 'ananskikh',
 'andreasyan',
 'andreev',
 'andreeva',
 'andronik',
 'anikeev',
 'anikeev',
 'anisimov',
 'anisimov',
 'anodina',
 'antipov',
 'antokhina',
 'antropenko',
 'anufrieva',
 'apazhev',
 'apeksimova',
 'arapov',
 'arefiev',
 'arenin',
 'aristov',
 'arkharov',
 'arshba',
 'arshinova',
 'artamnov',
 'artamonov',
 'artamonova',
 'artemov',
 'artyukhov',
 'asfandiyarov

I will add the name of each individual to the base url, than I start to acces each page and scrape for needed information like name, citizenship, date of birth, category, biography. Finally, create a dataframe with that information.

In [ ]:
base_url = "https://www.spisok-putina.org/en/personas/" # this is the base url

data = [] # for df creation

In [53]:
for name in persons_list:
    url = base_url + name # example of webpage: https://www.spisok-putina.org/en/personas/abramov/
    try:
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        
        # name
        name_element = soup.select_one('.material__title')
        name = name_element.text.strip() if name_element else ''
        print("Name:", name)
        
        # date of birth 
        dob_element = soup.find('strong', text="Date of Birth:")
        date_of_birth = dob_element.find_next('br').next_sibling.strip() if dob_element else ''
        print("Date of Birth:", date_of_birth)
        
        # citizenship
        citizenship_element = soup.find('strong', text="Citizenship:")
        citizenship = citizenship_element.find_next('br').next_sibling.strip() if citizenship_element else ''
        print("Citizenship:", citizenship)

        # categories (propagandist, etc.)
        categories_element = soup.find('strong', text="Categories:")
        categories = ""
        if categories_element:
            categories_tags = categories_element.find_next_siblings('a')
            categories_list = [tag.text.strip() for tag in categories_tags]
            categories = ', '.join(categories_list)

        print("Categories:", categories)

        # biography (only the first paragraph)
        biography_element = soup.find('div', class_='entry-content')
        for unwanted_section in biography_element.select('.section-accused, .section-materials'):
            unwanted_section.extract()

        biography_paragraph = biography_element.find('p')
        biography = biography_paragraph.text.strip() if biography_paragraph else ''
        print("Biography:", biography)

        # creating instances of each person
        data.append({
            'Name': name,
            'Date of Birth': date_of_birth,
            'Citizenship': citizenship,
            'Categories': categories,
            'Biography': biography
        })
    except requests.exceptions.RequestException:
        print("not found...")

df = pd.DataFrame(data)

Name: Abakarov Khizri  Magomedovich
Date of Birth: 28 June 1960
Citizenship: Russia
Categories: Beneficiaries
Biography: Russian statesman and public figure, entrepreneur. Head of the administration of the city of Derbent of the Republic of Dagestan. State Secretary of the Republic of Dagestan since November 10, 2020. Member of the State Duma of the Russian Federation since 2021 from the United Russia party


C:\Users\Asus\AppData\Local\Temp\ipykernel_3048\1693417587.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  dob_element = soup.find('strong', text="Date of Birth:")
C:\Users\Asus\AppData\Local\Temp\ipykernel_3048\1693417587.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  citizenship_element = soup.find('strong', text="Citizenship:")
C:\Users\Asus\AppData\Local\Temp\ipykernel_3048\1693417587.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  categories_element = soup.find('strong', text="Categories:")


Name: Abdullazyanov Edward Yunusovich
Date of Birth: 8 April 1957
Citizenship: Russia
Categories: Accomplices
Biography: Rector of Kazan State Power Engineering University.
Name: Abramov Ivan Nikolaevich
Date of Birth: 16 June 1978
Citizenship: Russia
Categories: Executors
Biography: Deputy Chairman of the Federation Council Committee on Economic Policy, representative of the executive body of state power of the Amur Region. Position before election (appointment): Deputy of the State Duma of the Federal Assembly of the Russian Federation of the seventh convocation, First Deputy Chairman of the State Duma Committee on Regional Policy and Problems of the North and the Far East.
Name: Абрамович Роман Аркадьевич
Date of Birth: 
Citizenship: 
Categories: 
Biography: Роман Абрамович родился 24 октября 1966 года в Саратове. В 4 года остался сиротой, воспитывал Романа его дядя Лейб Нахимович, работавший начальником рабочего снабжения лесной промышленности в Ухте. В 1974 году мальчик переехал в

Name: Aleinik Stanislav  Nikolaevich
Date of Birth: 10 June 1975
Citizenship: Russia
Categories: Accomplices
Biography: Rector of Belgorod State Agrarian University named after V.Ya. Gorin.
Name: Alekhin Andrey  Anatolyevich
Date of Birth: 9 February 1959
Citizenship: Russia
Categories: Executors
Biography: Russian politician, member of the Communist Party of the Russian Federation, deputy of the Legislative Assembly of the Omsk Region In 1994-2021, deputy of the State Duma of Russia since September 19, 2021. Won elections in a single-mandate constituency to the State Duma of the VIII convocation, the Communist Party of the Russian Federation.
Name: Алекперов Вагит Юсуфович
Date of Birth: 
Citizenship: 
Categories: 
Biography: с 1993 года по настоящий момент (2018 год) — президент ОАО “Лукойл”. С 2007 года — учредитель Фонда региональных социальных программ “Наше будущее”. С 2010 года — член Совета Фонда “Сколково”.
Name: Alekseenko Nikolai  Nikolaevich
Date of Birth: 29 November 1971


Name: Apazhev Aslan  Karalbievich
Date of Birth: 22 February 1974
Citizenship: Russia
Categories: Accomplices
Biography: Rector of the Kabardino-Balkarian State Agrarian University named after V.M. Kokova.
Name: Apeksimova Irina  Viktorovna
Date of Birth: 13 January 1966
Citizenship: Russia
Categories: Accomplices
Biography: Soviet and Russian actress, theater and film director, theater figure, singer, TV presenter. Director of the Taganka Theater and the Theater "Commonwealth of Taganka Actors". Honored Artist of the Russian Federation
Name: Arapov Georgy  Konstantinovich
Date of Birth: 11 September 1999
Citizenship: Russia
Categories: Executors
Biography: Russian politician, the youngest deputy of the State Duma of the Federal Assembly of the Russian Federation of the VIII convocation. Deputy Chairman of the Duma Committee on Ecology, Natural Resources and Environmental Protection "A Just Russia - For the Truth" food. party - New People.
Name: Arefiev Nikolai  Vasilyevich
Date of Bir

...

Name: Shubin Igor Nikolaevich
Date of Birth: 20 December 1955
Citizenship: Russia
Categories: Executors
Biography: Deputy of the State Duma of the 8th convocation, party - "United Russia".
Name: Shukshina Maria  Vasilievna
Date of Birth: 27 May 1967
Citizenship: Russia
Categories: Accomplices
Biography: Honored Artist of Russia, TV presenter, public figure.
Name: Shumakov Nikita Sergeevich
Date of Birth: 
Citizenship: 
Categories: Propagandists
Biography: General Producer of TVC.
Name: Shumilova Elena Borisovna
Date of Birth: 1 April 1978
Citizenship: Russia
Categories: Executors
Biography: Member of the Federation Council Committee on Federal Structure, Regional Policy, Local Self-Government and Northern Affairs, representative from the legislative (representative) body of state power of the Komi Republic. Position before election (appointment): PR-director of LLC " Luzales "; Deputy of the State Council of the Komi Republic of the seventh convocation.
Name: Shumkov Vadim  Mikhailovic

Name: Смолин Михаил Борисович
Date of Birth: 
Citizenship: 
Categories: 
Biography: Заместитель главного редактора «Царьград ТВ».
Name: Sobchak Ksenia
Date of Birth: 
Citizenship: Russia
Categories: Accomplices
Biography: Professional sphere of activity/official position/biography:
Name: Sobko Alexander
Date of Birth: 
Citizenship: 
Categories: Propagandists
Biography: Columnist at RIA Novosti.
Name: Sobolev Viktor Ivanovich
Date of Birth: 23 February 1950
Citizenship: Russia
Categories: Executors
Biography: Deputy of the State Duma of the 8th convocation, party - "Communist Party".
Name: Собянин Сергей Семёнович
Date of Birth: 
Citizenship: 
Categories: 
Biography: с 21 октября 2010 года по настоящий момент (2019 год) – мэр Москвы.
Name: Sokol Sergey Mikhailovich
Date of Birth: 17 December 1970
Citizenship: Russia
Categories: Executors
Biography: Deputy of the State Duma of the 8th convocation, party - "United Russia".
Name: Sokolov Maxim
Date of Birth: 10 September 1959
Citizenship: 

Name: Сусина Наталья Сергеевна
Date of Birth: 
Citizenship: 
Categories: 
Biography: Судья, выносившая заведомо неправосудные и политически мотивированные приговоры в отношении активистов оппозиции. Приговорила к реальным срокам нескольких фигурантов Болотного дела.
Name: Sverdlov Boris  Arkadievich
Date of Birth: 12 January 1954
Citizenship: Russia
Categories: Accomplices
Biography: Deputy Chairman of the Astrakhan regional public branch «Union of Writers of Russia».
Name: Svischev Dmitry Alexandrovich
Date of Birth: 22 May 1969
Citizenship: Russia
Categories: Executors
Biography: Deputy of the State Duma of the 8th convocation, party - LDPR.
Name: Svistunov Arkady Nikolaevich
Date of Birth: 28 April 1965
Citizenship: Russia
Categories: Executors
Biography: Deputy of the State Duma of the 8th convocation, party - LDPR.
Name: Svyatenko Inna Yuryevna
Date of Birth: 6 September 1967
Citizenship: Russia
Categories: Executors
Biography: Chairman of the Federation Council Committee on Socia

Name: 
Date of Birth: 
Citizenship: 
Categories: 


AttributeError: 'NoneType' object has no attribute 'select'

In [55]:
mydata = pd.DataFrame(data)

**There are some inaccuracies in data like for example some information are in russian, because there are duplicates (persons who share the same name) so his particular webpage is different. And, in the case of duplicates, I have scraped the russian version of webpage. So I will do some data transformation and preprocessing for adjusting these errors and gather as much information as I can.**

In [59]:
list_ = mydata['Name'].tolist()

#### Transforming ru chars names in English

In [65]:
english_first_words = []

for name in list_:
    name_parts = name.split() # split the name into words
    if len(name_parts) > 0: # check if the name has at least one word
        first_word = name_parts[0]
        if any(ord(char) > 128 for char in first_word): # check if the first word it's in Russian (if contains any non ASCII characters)
            english_first_word = translit(first_word, 'ru', reversed=True) # translate the first word from russian to en
            english_first_words.append(english_first_word.lower())

In [74]:
english_first_words

['abramovich',
 'afasizhev',
 'agalarov',
 'agramunt',
 'akimov',
 'akimov',
 'aksenov',
 'alekperov',
 'ammosov',
 'ananskih',
 'anodina',
 'antipov',
 'antropenko',
 'aristov',
 'artemov',
 'aslamova',
 'aulov',
 "avdot'ina",
 'aven',
 'averina',
 'babakov',
 'babajan',
 'baranov',
 'baranovskaja',
 'barsukov',
 'bastrykin',
 'beglov',
 'bezler',
 'bikchurin',
 'blavatnik',
 'bogdanchikov',
 'bogdanov',
 'bokarev',
 "bol'shunov",
 "bondar'",
 'bondarenko',
 'bondarenko',
 'borodin',
 'bortnikov',
 'brauer',
 'budilo',
 'bugynin',
 'burdin',
 'butina',
 'chalenko',
 'chemezov',
 'chepa',
 'chepiga',
 'cheprasova',
 'chernikova',
 'chernikova',
 'chernyshov',
 'churov',
 'danilkin',
 'danilov',
 'deljagin',
 'deripaska',
 'dimitriev',
 'dobkin',
 'dobrodeev',
 'dodon',
 'dodonova',
 'dolgareva',
 'drokova',
 'drymanov',
 'drymanov',
 "dudar'",
 'dugin',
 'dzhigurda',
 'egorova',
 'eremin',
 'ernst',
 'fadeev',
 'fadeev',
 'fedin',
 'fedorova',
 "fel'de",
 'firtash',
 'fradkov',
 'fradk

Some persons share the same name like for example the name "Ivanov". Then, the webpage coresponding to a particular person will be:
https://www.spisok-putina.org/personas/ivanov/, or https://www.spisok-putina.org/personas/ivanov-2/, or https://www.spisok-putina.org/personas/ivanov-3/ and so on. So I created a function to ad "-2" or "-3" (coressponding number), based on how many occurencies of that name are in the above list.

In [70]:
def transform_items(lst):
    item_count = {}
    transformed_list = []

    for item in lst:
        if item in item_count:
            item_count[item] += 1
        else:
            item_count[item] = 1

        count = item_count[item]

        if count == 1:
            transformed_list.append(f"{item}-2")
        else:
            transformed_list.append(f"{item}-{count}")

    return transformed_list

In [71]:
newlist_ = transform_items(english_first_words)

**Now, I can acces that pages too, saving a lot of information.**

In [72]:
newlist_

['abramovich-2',
 'afasizhev-2',
 'agalarov-2',
 'agramunt-2',
 'akimov-2',
 'akimov-2',
 'aksenov-2',
 'alekperov-2',
 'ammosov-2',
 'ananskih-2',
 'anodina-2',
 'antipov-2',
 'antropenko-2',
 'aristov-2',
 'artemov-2',
 'aslamova-2',
 'aulov-2',
 "avdot'ina-2",
 'aven-2',
 'averina-2',
 'babakov-2',
 'babajan-2',
 'baranov-2',
 'baranovskaja-2',
 'barsukov-2',
 'bastrykin-2',
 'beglov-2',
 'bezler-2',
 'bikchurin-2',
 'blavatnik-2',
 'bogdanchikov-2',
 'bogdanov-2',
 'bokarev-2',
 "bol'shunov-2",
 "bondar'-2",
 'bondarenko-2',
 'bondarenko-2',
 'borodin-2',
 'bortnikov-2',
 'brauer-2',
 'budilo-2',
 'bugynin-2',
 'burdin-2',
 'butina-2',
 'chalenko-2',
 'chemezov-2',
 'chepa-2',
 'chepiga-2',
 'cheprasova-2',
 'chernikova-2',
 'chernikova-2',
 'chernyshov-2',
 'churov-2',
 'danilkin-2',
 'danilov-2',
 'deljagin-2',
 'deripaska-2',
 'dimitriev-2',
 'dobkin-2',
 'dobrodeev-2',
 'dodon-2',
 'dodonova-2',
 'dolgareva-2',
 'drokova-2',
 'drymanov-2',
 'drymanov-2',
 "dudar'-2",
 'dugin-2'

Here I just used the same script as above

In [78]:
data2 = []

for name in newlist_:
    url = base_url + name
    try:
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        name_element = soup.select_one('.material__title')
        name = name_element.text.strip() if name_element else ''
        print("Name:", name)

        dob_element = soup.find('strong', text="Date of Birth:")
        date_of_birth = dob_element.find_next('br').next_sibling.strip() if dob_element else ''
        print("Date of Birth:", date_of_birth)

        citizenship_element = soup.find('strong', text="Citizenship:")
        citizenship = citizenship_element.find_next('br').next_sibling.strip() if citizenship_element else ''
        print("Citizenship:", citizenship)

        categories_element = soup.find('strong', text="Categories:")
        categories = ""
        if categories_element:
            categories_tags = categories_element.find_next_siblings('a')
            categories_list = [tag.text.strip() for tag in categories_tags]
            categories = ', '.join(categories_list)

        print("Categories:", categories)

        biography_element = soup.find('div', class_='entry-content')
        if biography_element:
            for unwanted_section in biography_element.select('.section-accused, .section-materials'):
                unwanted_section.extract()

            biography_paragraph = biography_element.find('p')
            biography = biography_paragraph.text.strip() if biography_paragraph else ''
            print("Biography:", biography)

            data2.append({
                'Name': name,
                'Date of Birth': date_of_birth,
                'Citizenship': citizenship,
                'Categories': categories,
                'Biography': biography
            })
        else:
            print("noo")
    except requests.exceptions.RequestException:
        print("noo")

df2 = pd.DataFrame(data2)

Name: Abramovich Roman
Date of Birth: 24 October 1966
Citizenship: Russia/Israel
Categories: Oligarchs and corrupt officials
Biography: Businessman


C:\Users\Asus\AppData\Local\Temp\ipykernel_3048\3245057743.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  dob_element = soup.find('strong', text="Date of Birth:")
C:\Users\Asus\AppData\Local\Temp\ipykernel_3048\3245057743.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  citizenship_element = soup.find('strong', text="Citizenship:")
C:\Users\Asus\AppData\Local\Temp\ipykernel_3048\3245057743.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  categories_element = soup.find('strong', text="Categories:")


Name: Afasizhev Zalim
Date of Birth: 
Citizenship: РФ
Categories: Executors, Law-destroyers
Biography: Investigator of the department for the investigation of especially important cases of the department for the North-West Administrative Office of the Main Investigative Directorate of the Investigative Committee in Moscow.
Name: Agalarov Araz lskender oglu
Date of Birth: 8 November 1955
Citizenship: Russia
Categories: Oligarchs and corrupt officials
Biography: President and Owner of Crocus Group
Name: Agramunt Pedro
Date of Birth: 12 September 1951
Citizenship: Kingdom of Spain
Categories: Beneficiaries
Biography: Russian-oriented politician, the European agent of Putin’s regime.
Former President of Parliamentary Assembly of the Council of Europe (PACE)
Since 1993 - Member of the Spanish Congress of Deputies, representing Valencian Autonomous Community.
Mr. Agaramunt holds an honorary degree Doctor Honoris Causa, given by the Russian Presidential Academy of National Economy and Public 

Name: Averina Arina Alekseyevna
Date of Birth: 13 August 2022
Citizenship: Russia
Categories: Accomplices
Biography: A Russian individual rhythmic gymnast. She is a 2020 Summer Olympics finalist, a two-time (2017, 2019) world all-around silver medalist, a two-time European all-around champion (2018, 2021) and the 2016 Grand Prix Final all-around bronze medalist. She is a three-time (2019-2021) Russian national all-around champion and a three-time (2015, 2017, 2022) Russian national all-around medalist. Her identical twin sister Dina Averina is also a competitive rhythmic gymnast.
Name: Babakov Alexander
Date of Birth: 
Citizenship: 
Categories: Executors, Oligarchs and corrupt officials
Biography: Deputy Chairman of the Federation Council Committee on Foreign Affairs, Special Representative of the President for Cooperation with Organizations of Compatriots Abroad.
Name: 
Date of Birth: 
Citizenship: 
Categories: 
No biography found for . Skipping...
Name: Baranov Oleg
Date of Birth: 16

Name: Dodon Igor
Date of Birth: 18 February 1975
Citizenship: 
Categories: Beneficiaries
Biography: President of Moldova
Name: Dodonova Tatiana
Date of Birth: 
Citizenship: Russia
Categories: Executors, Law-destroyers
Biography: DODONOVA, Tatiana Stanislavovna is a Judge of the Moscow City Court from February 2, 2013. She was a Judge of the Butyrsky District Court from November 17, 2008. There is not much publicly available information on Judge Dodonova. These are mainly reports of her participation in several political trials: the Bolotnaya Square case, the Yves Rocher case against Alexei Navalny, and a new one, the Moscow case.
Name: Dolgareva Anna
Date of Birth: 4 August 1988
Citizenship: 
Categories: Propagandists
Biography: Russian propagandist, "military commissar", "poetess"; uses pseudonyms Anna Lemert and Alonso Kexano.
Name: Drokova Maria
Date of Birth: 
Citizenship: 
Categories: Executors
Biography: Nashy Movement, federal commissioner
Name: Drymanov Alexander
Date of Birth:

...

Name: Lvova-Belova Maria
Date of Birth: 25 October 1984
Citizenship: Russia
Categories: Accomplices
Biography: Commissioner under the President of the Russian Federation for the rights of the child. One of the important (and the most notable) participants in the process of mass forcible removal of Ukrainian children from the occupied territories and their "re-education" in Russia.
Name: 
Date of Birth: 
Citizenship: 
Categories: 
No biography found for . Skipping...
Name: Magomedov Ziyavudin
Date of Birth: 25 September 1968
Citizenship: Russia
Categories: Oligarchs and corrupt officials
Biography: Owner of Summa Group, Member of the Russian Foreign Affairs Council
Name: Magomedov Ziyavudin
Date of Birth: 25 September 1968
Citizenship: Russia
Categories: Oligarchs and corrupt officials
Biography: Owner of Summa Group, Member of the Russian Foreign Affairs Council
Name: Makarov Andrei
Date of Birth: 22 July 1954
Citizenship: Russia
Categories: Executors, Propagandists
Biography: Member o

Name: Morozov Igor Nikolaevich
Date of Birth: 10 December 1956
Citizenship: Russia
Categories: Executors
Biography: Member of the Federation Council Committee on Economic Policy, representative of the legislative (representative) body of state power of the Ryazan region. Position before election (appointment): Deputy of the Ryazan City Duma of the seventh convocation.
Name: Морозов Игорь Николаевич
Date of Birth: 
Citizenship: 
Categories: 
Biography: Член Совета Федерации, представитель Рязанской области, член Комитета Совета Федерации по экономической политике. До избрания занимал должность депутата Рязанской городской Думы седьмого созыва.
Name: Morozov Oleg Viktorovich
Date of Birth: 5 November 1953
Citizenship: Russia
Categories: Executors
Biography: Deputy of the State Duma of the 8th convocation, party - "United Russia"
Name: Морозов Олег Викторович
Date of Birth: 
Citizenship: 
Categories: 
Biography: Депутат Государственной Думы 8-го созыва, партия - «Единая Россия».
Name: Mor

Name: Petrov Gennady
Date of Birth: 19 July 1947
Citizenship: Russia
Categories: Accomplices
Biography: Leader of the Tambov-Malyshev organized criminal group
Name: Petrov Gennady
Date of Birth: 19 July 1947
Citizenship: Russia
Categories: Accomplices
Biography: Leader of the Tambov-Malyshev organized criminal group
Name: Petrov Alexander Petrovich
Date of Birth: 21 May 1958
Citizenship: Russia
Categories: Executors
Biography: Deputy of the State Duma of the 8th convocation, party - "United Russia".
Name: Петров Александр Петрович
Date of Birth: 
Citizenship: 
Categories: 
Biography: Депутат Государственной Думы VIII созыва, партия - "Единая Россия".
Name: Petrov Yuri Alexandrovich
Date of Birth: 10 April 1947
Citizenship: Russia
Categories: Executors
Biography: Deputy of the State Duma of the 8th convocation, party - "United Russia".
Name: Петров Юрий Александрович
Date of Birth: 
Citizenship: 
Categories: 
Biography: Депутат Государственной Думы VIII созыва, партия - "Единая Россия".

Name: Rybolovlev Dmitry
Date of Birth: 22 November 1966
Citizenship: Russia/Cyprus
Categories: Oligarchs and corrupt officials
Biography: Russian oligarch, owner of FC Monaco
Name: Rykov Konstantin
Date of Birth: 27 May 1979
Citizenship: Russia
Categories: Executors, Propagandists
Biography: Russian Internet and media producer, founder of several online media, member of the State Duma of the 5th convocation from the United Russia party.
Name: Rylov Evgeny Mikhailovich
Date of Birth: 23 September 1996
Citizenship: Russia
Categories: Accomplices
Biography: Two-time Olympic champion in swimming.
Name: Ryzhenkin Leonid
Date of Birth: 10 November 1967
Citizenship: Russia
Categories: Executors
Biography: General Director of PJSC Mosttrest
Name: Ryzhova Anastasia
Date of Birth: 
Citizenship: Russia
Categories: Executors, Law-destroyers
Biography: RYZHOVA, Anastasia Vasilyevna was appointed judge of the Kaluga District Court of the Kaluga Region by presidential decree of July 15, 2000. In Augu

Name: 
Date of Birth: 
Citizenship: 
Categories: 
No biography found for . Skipping...
Name: Starikov Nikolay
Date of Birth: 23 August 1970
Citizenship: Russia
Categories: Propagandists
Biography: Propagandist
Name: Steshin Dmitry
Date of Birth: 12 September 1972
Citizenship: Russia
Categories: Propagandists
Biography: Special Correspondent of the Komsomolskaya Pravda Politics Department
Name: 
Date of Birth: 
Citizenship: 
Categories: 
No biography found for . Skipping...
Name: 
Date of Birth: 
Citizenship: 
Categories: 
No biography found for . Skipping...
Name: Strizhenova Ekaterina
Date of Birth: 20 March 1968
Citizenship: Russia
Categories: Propagandists
Biography: TV Presenter
Name: Surkov Vladislav
Date of Birth: 21 September 1964
Citizenship: Russia
Categories: Aggressors, Executors
Biography: Assistant to the President of Russia (2013 - to date), Deputy Prime Minister of the Russian Federation (2012 - 2013)
Name: Susina Natalya
Date of Birth: 
Citizenship: Russia
Categories: L

Now I have to remove the instances from dataframe that are in Russian and than concatanate it with df2, which is bassically the english translation of that deleted instances.

In [93]:
import unicodedata # this is for check if text contains ru characters

In [94]:
# check if text contains ru characters
def contains_russian(text):
    for char in text:
        if 'CYRILLIC' in unicodedata.name(char, ''):
            return True
    return False


# remove rows from the df that contain ru chars
def remove_rows_with_russian(df):
    indices_to_remove = []

    for index, row in df.iterrows():
        if any(contains_russian(str(cell)) for cell in row):
            indices_to_remove.append(index)

    cleaned_df = df.drop(indices_to_remove)
    return cleaned_df

In [95]:
without_ru = remove_rows_with_russian(mydata)

## Full dataset

In [101]:
full_dataset = pd.concat([without_ru, df2], ignore_index = True)

In [8]:
full_dataset.head()

,Name,Date of Birth,Citizenship,Categories,Biography
0,Abakarov Khizri Magomedovich,28 June 1960,Russia,Beneficiaries,"Russian statesman and public figure, entrepren..."
1,Abdullazyanov Edward Yunusovich,8 April 1957,Russia,Accomplices,Rector of Kazan State Power Engineering Univer...
2,Abramov Ivan Nikolaevich,16 June 1978,Russia,Executors,Deputy Chairman of the Federation Council Comm...
3,Afanasieva (Berg) Yulia,24 February 1988,Russia,Executors,An employee of the African Association for Fre...
4,Afanasov Mikhail Alexandrovich,15 July 1953,Russia,Executors,Member of the Committee of the Federation Coun...


In [22]:
full_dataset.describe().transpose()

,count,unique,top,freq
Name,1438,1286,Fedorov Yury Viktorovich,5
Date of Birth,1191,1038,22 January 1972,5
Citizenship,1301,15,Russia,1270
Categories,1398,24,Executors,635
Biography,1436,966,Deputy of the State Duma of the 8th convocatio...,76


In [18]:
full_dataset['Citizenship'].value_counts().head(3)

Russia             1270
Ukraine               9
Russia/Israel         3
Russia, Ukraine       3
Name: Citizenship, dtype: int64

In [20]:
full_dataset['Categories'].value_counts().head(8)

Executors                          635
Accomplices                        366
Propagandists                      161
Law-destroyers                      73
Oligarchs and corrupt officials     44
Beneficiaries                       35
Executors, Law-destroyers           22
Aggressors                          13
Name: Categories, dtype: int64